### Sign in with your Bentley account

In [ ]:
from evo.aio import AioTransport
from evo.common import APIConnector
from evo.oauth import AuthorizationCodeAuthorizer, EvoScopes, OAuthConnector

cache_location = "./notebook-data"

# Evo app credentials
client_id = "<your-client-id>"  # Replace with your client ID
redirect_url = "<your-redirect-url>"  # Replace with your redirect URL
org_id = "<your-organization-id>"  # Replace with your organization ID
hub_url = "<your-hub-url>"  # Replace with your Evo hub URL

# Service identity credentials
service_identity_email = "<your-service-identity-email>"  # Replace with your service identity email

client_name = "Evo Workspace API Examples"

transport = AioTransport(
    user_agent=client_name,
)

oauth_connector = OAuthConnector(
    transport=transport,
    client_id=client_id,
)

authorizer = AuthorizationCodeAuthorizer(
    oauth_connector=oauth_connector,
    redirect_url=redirect_url,
    scopes=EvoScopes.all_evo | EvoScopes.offline_access,
)

await authorizer.login()

### Create the API connector

In [ ]:
# The API connector can be used to make direct API calls
connector = APIConnector(hub_url, transport, authorizer)

### List all workspaces as an Evo admin

In [ ]:
import json
from http import HTTPStatus

import pandas as pd

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/admin/orgs/{org_id}/workspaces"

    path_params = {
        "org_id": org_id,
    }

    api_response = await connector.call_api(
        method="GET",
        resource_path=resource_path,
        path_params=path_params,
        response_types_map={
            "200": HTTPResponse,
        },
    )

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Display as a table
    workspace_list = [
        {
            "Name": ws["name"],
            "ID": ws["id"],
        }
        for ws in response_json["results"]
    ]

    df = pd.DataFrame(workspace_list)
    display(df)
except Exception as e:
    print(f"Error: User is not an Evo admin\n{e}")

### Add service account to the Evo instance

In [ ]:
import json

from pygments import highlight
from pygments.formatters import TerminalTrueColorFormatter
from pygments.lexers import JsonLexer

from evo.common.data import HTTPResponse

try:
    resource_path = "/workspace/orgs/{org_id}/members"

    path_params = {
        "org_id": org_id,
    }

    body = {
        "users": [
            {
                "email": service_identity_email,
                "roles": ["de402ff1-058e-4685-b4d4-9fe846270279"],  # ID for the 'Evo User' role
            }
        ]
    }

    api_response = await connector.call_api(
        method="POST",
        resource_path=resource_path,
        path_params=path_params,
        body=body,
        header_params={"Content-Type": "application/json"},
        response_types_map={
            "201": HTTPResponse,
        },
    )

    # Parse the response data
    response = api_response.data.decode("utf-8")
    response_json = json.loads(response)

    # Print the response in a highlighted format
    print(highlight(json.dumps(response_json, indent=4), JsonLexer(), TerminalTrueColorFormatter(style="lightbulb")))

    service_identity_id = response_json["members"][0]["id"]
    print(f"Service identity added with ID: {service_identity_id}")

except Exception as e:
    print(f"Error: Failed to add service identity\n{e}")

### Add service account to the Evo workspace

The final step is to add the service identity to a given workspace.
Before running this cell, be sure to copy:
- The ID of a workspace from the table of workspaces
- The ID of the service identity from the API response in the previous cell

In [ ]:
from evo.common.data import HTTPResponse

workspace_id = "<your-workspace-id>"  # Replace with your workspace ID

try:
    resource_path = "/workspace/admin/orgs/{org_id}/workspaces/{workspace_id}/users"

    path_params = {
        "org_id": org_id,
        "workspace_id": workspace_id,
    }

    body = {
        "user_id": service_identity_id,
        "email": service_identity_email,
        "role": "editor",  # Not recommended to use "owner" for service identities
    }

    api_response = await connector.call_api(
        method="POST",
        resource_path=resource_path,
        path_params=path_params,
        header_params={"Content-Type": "application/json"},
        body=body,
        response_types_map={
            "201": HTTPResponse,
        },
    )

    status = api_response.status
    if status == HTTPStatus.CREATED:
        print("Service identity added to workspace successfully.")

except Exception as e:
    print(f"Error assigning user role:\n{e}")